In [1]:
!pip install pathway bokeh panel --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import datetime
import pathway as pw
import bokeh.plotting as bk
import panel as pn
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
from bokeh.plotting import figure, curdoc
from bokeh.io import output_notebook, push_notebook, show


In [3]:
from google.colab import files
uploaded = files.upload()


Saving dataset.csv to dataset.csv


In [5]:
df = pd.read_csv('dataset.csv')  # Upload this file manually in Colab


In [8]:
df['timestamp'] = pd.to_datetime(df['LastUpdatedTime'])  # or whatever the real name is


In [9]:
df.sort_values(by='timestamp', inplace=True)
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,timestamp
7370,7370,BHMNCPNST01,485,26.140048,91.730972,215,car,low,2,1,20-11-2016,07:41:00,2025-07-09 07:41:00
7518,7518,BHMNCPNST01,485,26.140048,91.730972,193,bike,low,1,0,28-11-2016,07:42:00,2025-07-09 07:42:00
7514,7514,BHMNCPNST01,485,26.140048,91.730972,193,car,low,1,0,28-11-2016,07:42:00,2025-07-09 07:42:00
7515,7515,BHMNCPNST01,485,26.140048,91.730972,193,truck,low,2,0,28-11-2016,07:42:00,2025-07-09 07:42:00
7516,7516,BHMNCPNST01,485,26.140048,91.730972,193,car,low,2,0,28-11-2016,07:42:00,2025-07-09 07:42:00


In [10]:
def model1_linear_price(prev_price, occupancy, capacity, alpha=2):
    occ_rate = occupancy / capacity
    new_price = prev_price + alpha * occ_rate
    return max(5, min(20, new_price))


In [11]:
def model2_demand_price(base_price, row, weights, lambd=0.2):
    occ = row['occupancy']
    cap = row['capacity']
    queue = row['queue_length']
    traffic = row['traffic_level']
    is_special = row['special_day']
    vehicle_type = row['vehicle_type']  # Assume mapping done before

    demand = (
        weights['occ'] * (occ / cap) +
        weights['queue'] * queue -
        weights['traffic'] * traffic +
        weights['special'] * is_special +
        weights['vehicle'] * vehicle_type
    )

    norm_demand = (demand - 0) / (10)  # assume max demand ~ 10
    price = base_price * (1 + lambd * norm_demand)
    return max(5, min(20, price))


In [13]:
from geopy.distance import geodesic

def get_nearby_lots(current_lot_id, df_meta, radius_m=500):
    """
    Returns a list of nearby lot_ids within radius in meters.
    """
    current = df_meta[df_meta['lot_id'] == current_lot_id].iloc[0]
    current_coords = (current['latitude'], current['longitude'])

    nearby = []
    for _, row in df_meta.iterrows():
        if row['lot_id'] == current_lot_id:
            continue
        other_coords = (row['latitude'], row['longitude'])
        distance = geodesic(current_coords, other_coords).meters
        if distance <= radius_m:
            nearby.append(row['lot_id'])
    return nearby


In [14]:
def model3_competitive_price(base_price, row, current_time, all_prices_df, df_meta, weights, lambd=0.2):
    """
    Determines new price based on own demand + competitor prices.
    """
    occ = row['occupancy']
    cap = row['capacity']
    queue = row['queue_length']
    traffic = row['traffic_level']
    is_special = row['special_day']
    vehicle_type = row['vehicle_type']
    lot_id = row['lot_id']

    # --- Demand based base price (same as Model 2) ---
    demand = (
        weights['occ'] * (occ / cap) +
        weights['queue'] * queue -
        weights['traffic'] * traffic +
        weights['special'] * is_special +
        weights['vehicle'] * vehicle_type
    )
    norm_demand = (demand - 0) / 10
    my_price = base_price * (1 + lambd * norm_demand)

    # --- Competitor logic ---
    nearby_lots = get_nearby_lots(lot_id, df_meta)
    recent_prices = all_prices_df[
        (all_prices_df['timestamp'] == current_time) &
        (all_prices_df['lot_id'].isin(nearby_lots))
    ]

    if not recent_prices.empty:
        avg_comp_price = recent_prices['price'].mean()
        if occ >= cap and my_price > avg_comp_price:
            # We're full & more expensive → reduce price slightly
            my_price = max(base_price, avg_comp_price - 1)
        elif avg_comp_price > my_price:
            # Competitors are more expensive → we can increase price
            my_price = min(my_price + 1, 20)

    return max(5, min(20, my_price))


In [15]:
def simulate_model3(df, df_meta, base_price=10):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['LastUpdatedTime'])
    df.sort_values(by='timestamp', inplace=True)

    prices = []
    all_prices_df = pd.DataFrame(columns=['timestamp', 'lot_id', 'price'])

    weights = {
        'occ': 1,
        'queue': 0.5,
        'traffic': 0.7,
        'special': 0.8,
        'vehicle': 0.4
    }

    for idx, row in df.iterrows():
        current_time = row['timestamp']
        lot_id = row['lot_id']
        price = model3_competitive_price(
            base_price, row, current_time,
            all_prices_df, df_meta, weights
        )
        prices.append(price)

        all_prices_df = pd.concat([
            all_prices_df,
            pd.DataFrame([{'timestamp': current_time, 'lot_id': lot_id, 'price': price}])
        ])

    df['price'] = prices
    return df


In [16]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

def plot_prices(df, lot_id_filter):
    filtered = df[df['lot_id'] == lot_id_filter]
    source = ColumnDataSource(filtered)

    p = figure(x_axis_type='datetime', title=f'Price Over Time – Lot {lot_id_filter}')
    p.line(x='timestamp', y='price', source=source, line_width=2)
    show(p)
